# Homework 4: Sentence Segmentation

Sentence segmentation is a crucial, yet under-appreciated, part of any text processing pipeline. In any real-world setting, text will not come to you pre-chunked into sentences, and if your pipeline involves any sentence-level analysis (parsing, translation, entity & co-reference extraction, etc.) the first thing you'll need to do will be split things up into sentences. This is a deceptively challenging problem, as punctuation can be used in multiple ways. Consider the following sentence (borrowed from Kyle Gorman's [excellent discussion of the subject](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/)): 

> _"Rolls-Royce Motor Cars Inc. said it expects its U.S. sales to remain steady at about 1,200 cars in 1990."_ 

This sentence features several periods, but only one of them (the last one) represents a valid sentence break. Sentences can also have embedded clauses or quotation marks, some of which may superficially look like a sentence boundary: 

> _He says the big questions–“Do you really need this much money to put up these investments? Have you told investors what is happening in your sector? What about your track record?–“aren’t asked of companies coming to market._

There are many ways to approach this particular problem; it can be approached using rules, which themselves may be encoded as regular expressions, or (more usefully) it can be modeled as a machine learning problem. Specifically, as a binary classification problem, where candidate sentence boundaries (i.e., punctuation marks from the closed class of English punctuation that can represent a sentence boundary) are either _positive_ examples (actual sentence boundaries) or _false_ (not sentence boundaries). 

In this assignment, you will develop and test your own sentence boundary detection algorithm. This assignment is shorter than our previous assignments, and is also much more open-ended. By this point in the class, you have been exposed to several different families of technique for text classification and analysis, and have hopefully begun to build up some intuitions. For this assignment, you can choose whatever methods you like: log-linear models, rule-based methods, some sort of neural network, it's up to you!

## 0. Setup & data exploration

In [167]:
from importlib import reload
from sklearn.metrics import classification_report, accuracy_score

In [168]:
from hw4_utils import data
from hw4_utils import classifier

We've provided a version of the WSJ section of the Penn Treebank to use for training and evaluation purposes. See `data/wsj_sbd/README.md` for a description of where this data came from and how it was assembled.

The `hw4_utils.data` module has a useful method for reading in data (`file_as_string`) and another utility function to extract candidate boundaries. Let's use these to explore our data a little bit.

In [169]:
dev = data.file_as_string("data/wsj_sbd/dev.txt")
train = data.file_as_string("data/wsj_sbd/train.txt")
test = data.file_as_string("data/wsj_sbd/test.txt")

In [170]:
dev[:105]

'Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29.\nMr. Vinken is chairm'

Each of these is a string containing the text, with one input sentence per line. Note that newline characters are still present, which will be important as this will provide us with our ground truth.

Remember, we _only_ will use the `test` split for our final evaluation. For data exploration, feature engineering, and for tuning hyperparameters, use the `dev` split.

The first and most important thing to find out when doing classification is what your class probabilities are, so let's look into that. The `load_candidates()` function in the `hw4_utils.data` module will read an input data split, identify candidate boundaries, and extract information about the candidates context that we may want to use for feature engineering. One of the attributes it will extract is whether or not the candidate is actually a true sentence break (`is_true_break`). Obviously, this would only be useful at training time, as in the "real world" we wouldn't know the answer. :-)

We will use `load_candidates()` to answer our question about class balance:

In [171]:
from collections import Counter

Counter([c.is_true_break for c in data.load_candidates(dev)])

Counter({False: 1766, True: 5769})

As is often the case, our classes are quite imbalanced! Let's load in our ground truth labels:

In [172]:
y_dev = [o.is_true_break for o in data.load_candidates(dev)]

What else can we find out about our candidate breaks?

In [173]:
candidates = list(data.load_candidates(dev[:10000])) # for now, just get candidates from the first 10k characters
first_can = candidates[6]
print(candidates[:1])

[Observation(left_token='Nov', left_raw='d as a nonexecutive director Nov', punctuation_mark='.', right_token='29', right_raw='29. Mr. Vinken is chairman of El', is_true_break=False, end_offset=81, orig_obs='d as a nonexecutive director Nov. 29. Mr. Vinken is chairman of El')]


In [174]:
print("original context: ",first_can.orig_obs)
print("punctuation mark: ", first_can.punctuation_mark)
print("token to the left: ", first_can.left_token)
print("token to the right: ", first_can.right_token)
print("token to the right: ", first_can.end_offset)

original context:   decades later, researchers said. Lorillard Inc., the unit of New 
punctuation mark:  .
token to the left:  said
token to the right:  Lorillard
token to the right:  678


Take a look at the the `Observation` namedtuple in `hw4_utils.data` for more details about the various attributes that you can find out about each possible sentence boundary.

## 1. Baselines

We will begin by looking at three different baselines. First, we will 

### Baseline #1: Majority class

Since our data are quite imbalanced, what if we always treat every candidate boundary as though it was a true boundary?

In [175]:
y_hat_baseline = [classifier.baseline_classifier(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_baseline))

             precision    recall  f1-score   support

      False       0.00      0.00      0.00      1766
       True       0.77      1.00      0.87      5769

avg / total       0.59      0.77      0.66      7535



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Baseline #2: Next-token capitalization

What if we say that a candidate is a boundary if the following token is capitalized?

In [176]:
y_hat_next_tok_cap = [classifier.next_tok_capitalized_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_next_tok_cap))

             precision    recall  f1-score   support

      False       0.60      0.40      0.48      1766
       True       0.83      0.92      0.87      5769

avg / total       0.78      0.80      0.78      7535



### Baseline #3: Punkt

Using the [NLTK implementation](http://www.nltk.org/_modules/nltk/tokenize/punkt.html) of [Punkt (Kiss & Strunk, 2006)](https://www.mitpressjournals.org/doi/pdf/10.1162/coli.2006.32.4.485):

(Note: You'll need to have downloaded the NLTK pre-trained Punkt model for this baseline to work)

In [177]:
y_hat_punkt = [classifier.punkt_baseline(o) for o in data.load_candidates(data.file_as_string("data/wsj_sbd/dev.txt"))]
print(classification_report(y_dev, y_hat_punkt))

             precision    recall  f1-score   support

      False       0.92      0.43      0.59      1766
       True       0.85      0.99      0.91      5769

avg / total       0.87      0.86      0.84      7535



### Conclusion

None of these are especially inspiring in their performance... can you do better?

## 2. Your turn!

### Part 1: Now it's your turn.

Your mission, in this assignment, is to implement at least two additional approaches. The approaches must be different in terms of the features you choose to use, classification approach, or both, but beyond that limitation you are free to do whatever you like as long as it is your own work (i.e., do not download and use an already-existing sentence boundary detector tool). You may, however, replicate an existing algorithm, though please be very careful not about re-using other people's code. 

For a discussion of features that may prove useful, and of other approaches to this task that you may wish to replicate or build off of, consult the bibliography discussed in [Gorman (2014)](http://www.wellformedness.com/blog/simpler-sentence-boundary-detection/); note that the specific sentence boundary detector described on that page uses a small set of highly useful features, but you will want to go beyond simply replicating his system.

***Deliverable***: The code for your ≥ two additional classifiers, along with a brief writeup (≈1 page) of what you built and why you chose those specific approaches and features. 

### Part 2: Evaluation & Error analysis

After building and describing your additional classifiers, evaluate their performance against the `test` partition of the data as shown above. Write a brief summary of their relative performance, and include an _error analysis_. Examine cases where your classifiers failed to predict correctly, or disagreed with one another, and see if you can identify any patterns that might explain where they were going awry.

***Deliverable***: Your written summary, as well as any relevant data tables.

In [216]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score, roc_curve, auc, precision_score
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, svm
import nltk
%matplotlib inline

In [262]:
#Features: length of L(LOL),is R uppercase, L contains period(LP), POS of L(POL), POS of R(POR)
from hw4_utils import data
from hw4_utils import classifier

y_train = [o.is_true_break for o in data.load_candidates(train)]
X_train = list(data.load_candidates(train))

X_dev = list(data.load_candidates(dev))

X_test = list(data.load_candidates(test))

data = []

def is_first_up(word):
    if word.isalpha():
        return word[0].isupper()
    return False

def cont_period(word):
    if "." in word:
        return True
    return False

def is_ellipse(word):
    count = 0
    for c in word:
        if c == ".":
            count += 1
        else:
            if count >= 1:
                return bool(count)
            else:
                count = 0
    return bool(count)
train_set_tags = Counter()

for o in X_train:
    example = []
    example.append(len(o.left_token))
    example.append(is_first_up(o.right_token))
    example.append(cont_period(o.left_token))
    example.append(nltk.pos_tag([o.left_token])[0][1])
    example.append(nltk.pos_tag([o.right_token])[0][1])
    data.append(example)
    
for o in X_dev:
    example = []
    example.append(len(o.left_token))
    example.append(is_first_up(o.right_token))
    example.append(cont_period(o.left_token))
    example.append(nltk.pos_tag([o.left_token])[0][1])
    example.append(nltk.pos_tag([o.right_token])[0][1])
    data.append(example)
    
for o in X_test:
    example = []
    example.append(len(o.left_token))
    example.append(is_first_up(o.right_token))
    example.append(cont_period(o.left_token))
    example.append(nltk.pos_tag([o.left_token])[0][1])
    example.append(nltk.pos_tag([o.right_token])[0][1])
    data.append(example)

    
df = pd.DataFrame(data,columns=['LOL','IRU','LP','POL','POR'])
df.head()

#print(train_set_tags)

,LOL,IRU,LP,POL,POR
0,9,True,False,NN,NNP
1,4,True,False,NN,DT
2,5,True,False,NN,DT
3,8,True,False,NN,DT
4,4,True,False,NN,CC


In [263]:
labelencoder = preprocessing.LabelEncoder()
df.iloc[:,1] = labelencoder.fit_transform(df.iloc[:,1])
df.iloc[:,2] = labelencoder.fit_transform(df.iloc[:,2])
df.iloc[:,3] = labelencoder.fit_transform(df.iloc[:,3])
df.iloc[:,4] = labelencoder.fit_transform(df.iloc[:,4])

onehotencoder = preprocessing.OneHotEncoder(categorical_features = [1,2,3,4])
data = onehotencoder.fit_transform(df).toarray()

In [264]:
X_train = data[:len(y_train),:]
X_dev = data[len(y_train):len(y_train)+len(y_dev),:]

In [265]:
X_train_scaled = preprocessing.scale(X_train)
scaler = preprocessing.StandardScaler().fit(X_train)
X_dev_scaled = scaler.transform(X_dev)

clf = LinearSVC()
clf.fit(X_train_scaled, y_train)
y_svm = clf.predict(X_dev_scaled)
print(classification_report(y_dev, y_svm))

             precision    recall  f1-score   support

      False       0.93      0.95      0.94      1766
       True       0.99      0.98      0.98      5769

avg / total       0.97      0.97      0.97      7535



#### Error analysis

In [266]:
from hw4_utils import data
from hw4_utils import classifier
for i,o in enumerate(data.load_candidates(dev)):
  if y_svm[i] != y_dev[i]:
    print(o.orig_obs, 'has been classified as ', y_svm[i], 'and should be ', y_dev[i])

xample, rose to 8.04% from 7.90%. Despite recent declines in yield has been classified as  False and should be  True
 currently yielding well over 9%. Dreyfus World-Wide Dollar, the t has been classified as  False and should be  True
d to an average 8.53% from 8.56%. J.P. Bolduc, vice chairman of W. has been classified as  False and should be  True
s Operations to Finmeccanica S.p. A. for $295 million. Finmeccanic has been classified as  True and should be  False
s were at $50.38 billion, up 19%. Newsweek, trying to keep pace wi has been classified as  False and should be  True
of 4,393,237, a decrease of 7.3%. Newsweek's circulation for the f has been classified as  False and should be  True
t from the same period last year. U.S. News' circulation in the sa has been classified as  False and should be  True
me time was 2,303,328, down 2.6%. New England Electric System bowe has been classified as  False and should be  True
nterest rate on the refund at 9%. Commonwealth Edison now faces 

re. Source: Fulton Prebon (U.S.A.) Inc. DISCOUNT RATE: 7%. The char has been classified as  True and should be  False
ents for delivery within 30 days. 9.82%, standard conventional fix has been classified as  False and should be  True
 LYNCH READY ASSETS TRUST: 8.64%. Annualized average rate of retur has been classified as  False and should be  True
etimes you just go with your gut." Mr. Bernstein said he will stay  has been classified as  False and should be  True
tments as provided by Article II. Article II places on the preside has been classified as  False and should be  True
ss appointments under Article II. Section 605 also imposes unconst has been classified as  False and should be  True
 1, Buick sales have plunged 33%. For American Express, the promot has been classified as  False and should be  True
ew card holders realize this, Mr. Riese says. Until now, however,  has been classified as  True and should be  False
sn't carry any finance rates. Mr. Riese says American Express 

ays an official of the Japan-U.S. Business Council. But to Mitsubi has been classified as  True and should be  False
or 30 years and six trillion yen. Time Warner Inc. and Sony Corp.  has been classified as  False and should be  True
er senior vice president Jacob F. "Jake" Horton was the mastermind has been classified as  True and should be  False
 II Inc. and Hemmer & Yates Corp. -- to make payments to various p has been classified as  True and should be  False
ual sales gains of more than 10%. To increase their share of that  has been classified as  False and should be  True
 products, including Tide and Mr. Clean, in Canada, but doesn't pl has been classified as  True and should be  False
't plan to bring them to the U.S.. Marketers believe most Americans has been classified as  False and should be  True
be less there than meets the eye. Mr. Moon planned to convert mill has been classified as  False and should be  True
cation Church members in the U.S. Mr. Moon's support for a Wate

In [232]:
import string
def has_verb(s):
    for t in list(nltk.pos_tag(s.split())):
        w,pos = t
        if pos == "VB":
            return True
    return False

def is_left_punct(w):
    if w in string.punctuation:
        return True
    return False

def is_right_num(w):
    try:
        float(w)
        return True
    except ValueError:
        return False
    
# New features: Left raw has verb(LOV), Right raw has verb(ROV), Is Left Punct(ILP), Is Right Number(IRN), is L uppercase(ILU)

##### Experiment 2

In [284]:
from hw4_utils import data
from hw4_utils import classifier

y_train = [o.is_true_break for o in data.load_candidates(train)]
X_train = list(data.load_candidates(train))

X_dev = list(data.load_candidates(dev))

X_test = list(data.load_candidates(test))

data = []

for o in X_train:
    example = []
    example.append(len(o.left_token))
    example.append(is_first_up(o.right_token))
    example.append(cont_period(o.left_token))
    example.append(nltk.pos_tag([o.left_token])[0][1])
    example.append(nltk.pos_tag([o.right_token])[0][1])
    example.append(has_verb(o.left_raw))
    example.append(has_verb(o.right_raw))
    example.append(is_left_punct(o.left_token))
    example.append(is_right_num(o.right_token))
    example.append(is_first_up(o.left_token))
    data.append(example)
    
for o in X_dev:
    example = []
    example.append(len(o.left_token))
    example.append(is_first_up(o.right_token))
    example.append(cont_period(o.left_token))
    example.append(nltk.pos_tag([o.left_token])[0][1])
    example.append(nltk.pos_tag([o.right_token])[0][1])
    example.append(has_verb(o.left_raw))
    example.append(has_verb(o.right_raw))
    example.append(is_left_punct(o.left_token))
    example.append(is_right_num(o.right_token))
    example.append(is_first_up(o.left_token))
    data.append(example)
    
for o in X_test:
    example = []
    example.append(len(o.left_token))
    example.append(is_first_up(o.right_token))
    example.append(cont_period(o.left_token))
    example.append(nltk.pos_tag([o.left_token])[0][1])
    example.append(nltk.pos_tag([o.right_token])[0][1])
    example.append(has_verb(o.left_raw))
    example.append(has_verb(o.right_raw))
    example.append(is_left_punct(o.left_token))
    example.append(is_right_num(o.right_token))
    example.append(is_first_up(o.left_token))
    data.append(example)

    
df = pd.DataFrame(data,columns=['LOL','IRU','LP','POL','POR', 'LOV','ROV','ILP','IRN','ILU'])
df.head()

,LOL,IRU,LP,POL,POR,LOV,ROV,ILP,IRN,ILU
0,9,True,False,NN,NNP,False,False,False,False,False
1,4,True,False,NN,DT,False,False,False,False,False
2,5,True,False,NN,DT,False,True,False,False,False
3,8,True,False,NN,DT,False,False,False,False,False
4,4,True,False,NN,CC,False,False,False,False,False


In [285]:
labelencoder = preprocessing.LabelEncoder()
df.iloc[:,1] = labelencoder.fit_transform(df.iloc[:,1])
df.iloc[:,2] = labelencoder.fit_transform(df.iloc[:,2])
df.iloc[:,3] = labelencoder.fit_transform(df.iloc[:,3])
df.iloc[:,4] = labelencoder.fit_transform(df.iloc[:,4])
df.iloc[:,5] = labelencoder.fit_transform(df.iloc[:,5])
df.iloc[:,6] = labelencoder.fit_transform(df.iloc[:,6])
df.iloc[:,7] = labelencoder.fit_transform(df.iloc[:,7])
df.iloc[:,8] = labelencoder.fit_transform(df.iloc[:,8])

onehotencoder = preprocessing.OneHotEncoder(categorical_features = [1,2,3,4,5,6,7,8])
data = onehotencoder.fit_transform(df).toarray()

In [286]:
X_train = data[:len(y_train),:]
X_dev = data[len(y_train):len(y_train)+len(y_dev),:]
X_test = data[len(y_train)+len(y_dev):,:]

In [276]:
X_train_scaled = preprocessing.scale(X_train)
scaler = preprocessing.StandardScaler().fit(X_train)
X_dev_scaled = scaler.transform(X_dev)

clf = LinearSVC()
clf.fit(X_train_scaled, y_train)
y_svm = clf.predict(X_dev_scaled)
print(classification_report(y_dev, y_svm))

             precision    recall  f1-score   support

      False       0.96      0.98      0.97      1766
       True       0.99      0.99      0.99      5769

avg / total       0.98      0.98      0.98      7535



In [277]:
clf = svm.SVC(kernel='rbf',gamma=1,C=1)
clf.fit(X_train_scaled, y_train)
y_svm = clf.predict(X_dev_scaled)
print(classification_report(y_dev, y_svm))

             precision    recall  f1-score   support

      False       0.98      0.96      0.97      1766
       True       0.99      1.00      0.99      5769

avg / total       0.99      0.99      0.99      7535



#### Error analysis

In [246]:
from hw4_utils import data
from hw4_utils import classifier
for i,o in enumerate(data.load_candidates(dev)):
  if y_svm[i] != y_dev[i]:
    print(o.orig_obs, 'classified as ', y_svm[i], 'should be ', y_dev[i])

s Operations to Finmeccanica S.p. A. for $295 million. Finmeccanic classified as  True should be  False
e said. "That attracts attention... it was just another one of the r classified as  False should be  True
be paid $240,000. Besides Messrs. Cray and Barnum, other senior ma classified as  True should be  False
eveloped the disk drives for PCs. Dennis Hayes and Dale Heathering classified as  False should be  True
me $38.3 billion world-wide. F.H. Faulding & Co., an Australian ph classified as  True should be  False
imex Inc. for changes in the U.S. Generalized System of Preference classified as  True should be  False
 other investors outside the U.S.." Such devices have boosted Japane classified as  False should be  True
ional Monetary Fund. The U.S.S.R. belongs to neither organization. classified as  True should be  False
Jim Courter. "Remember Pinocchio?" says a female voice. "Consider J classified as  True should be  False
 candidates. "Who's really lying?" asks a female voice. "Fl

ces under a bill entered by Reps. Chandler (R., Wash.) and Andrews classified as  True should be  False
 said, `Why don't you come to us?' '' Mr. Spielvogel said. Mr. Ache classified as  True should be  False
backed Cambodian leader} Hun Sen. Is the trouble over? Can Sihanou classified as  False should be  True
here does that first stimulus go?" exclaims SUNY neurologist Paul M classified as  True should be  False


In [287]:
clf = svm.SVC(kernel='rbf',gamma=1.5,C=2)
clf.fit(X_train_scaled, y_train)

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1.5, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [288]:
from hw4_utils import data
from hw4_utils import classifier

X_test_scaled = scaler.transform(X_test)
y_svm = clf.predict(X_test_scaled)
y_test = [o.is_true_break for o in data.load_candidates(test)]
print(classification_report(y_test, y_svm))

             precision    recall  f1-score   support

      False       0.99      0.97      0.98      2205
       True       0.99      1.00      0.99      7531

avg / total       0.99      0.99      0.99      9736

